<a href="https://colab.research.google.com/github/liuyixin-louis/dl_learning/blob/master/Testing_DCP_FLOPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#安装相关包

In [0]:
!git clone https://github.com/SCUT-AILab/DCP
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzvf /content/cifar-10-python.tar.gz
!pip install pyhocon
!pip install prettytable
!pip install torch==0.4.1
!pip install torchvision==0.2.0
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!wget https://github.com/SCUT-AILab/DCP/releases/download/0.1/resnet_56_pruned0.5.pth

#设置参数

In [2]:
import torch
torch.cuda.device_count()

1L

In [0]:
import sys
sys.path.append('/content/DCP')
sys.path.append('/content/DCP/')
import os 
path = '/content/DCP/dcp/test'
os.chdir(path)
%tensorflow_version 1.x
os.environ["CUDA_VISIBLE_DEVICES"]='0'

## test.hocon配置：

### **test.hocon**

```
# ------------- general options ----------------------------------------
save_path = "/content/DCP/dcp/test/" # log path
data_path = "/content/cifar-10-batches-py/" # path for dataset folder, containing cifar10 and imagenet
dataset = "cifar10"  # options: imagenet | cifar10
seed = 1 # manually set RNG seed
gpu = "0"  # GPU id to use, e.g. "0,1,2,3"
print_frequency = 10

# ------------- data options -------------------------------------------
n_threads = 4  # number of threads used for data loading
n_classes = 10  # number of classes in the dataset

# ------------- discrimination-aware options ---------------------------
pruning_rate = 0.5  # pruning rate

# ------------- common optimization options ----------------------------
batch_size = 128  # mini-batch size
momentum = 0.9  # momentum
weight_decay = 1e-4  # weight decay
lr = 0.1  # initial learning rate
n_epochs = 1 # number of total epochs
step = [160, 240]  # multi-step for linear learning rate

# ------------- model options ------------------------------------------
net_type = "preresnet"  # options: resnet | preresnet
experiment_id = "evaluation_20200522 "  # experiment identifier
depth = 56# resnet depth: (n-2)%6==0

# ------------- resume or pretrained options ------------------------------
pretrained = "/content/resnet_56_pruned0.5.pth"  # path of the pre-trained model
resume = "" # resume checkpoint
```



#计算FLOPs

In [0]:
import argparse
import torch.backends.cudnn as cudnn
import torch.utils.data
import torch.utils.data.distributed
from option import Option
from dcp.pruning import ResModelPrune
from torch import nn

from dcp.trainer import Trainer
from dcp.aux_checkpoint import AuxCheckPoint
from dcp.dataloader import *
from dcp.mask_conv import MaskConv2d
from dcp.model_builder import get_model
from dcp.models.preresnet import PreBasicBlock
from dcp.models.resnet import BasicBlock, Bottleneck
from dcp.utils.logger import get_logger
from dcp.utils.model_analyse import ModelAnalyse
from dcp.utils.tensorboard_logger import TensorboardLogger
from dcp.utils.write_log import write_log, write_settings
from dcp.utils import cal_pivot
from dcp.models.pruned_preresnet import PrunedPreResNet
from dcp.models.pruned_resnet import PrunedResNet
from dcp.models.pruned_vgg import PrunedVGGCIFAR


class Experiment(object):
    """
    Run experiments with pre-defined pipeline
    """

    def __init__(self, options=None, conf_path=None):
        self.settings = options or Option(conf_path)
        self.checkpoint = None
        self.train_loader = None
        self.val_loader = None
        self.pruned_model = None
        self.network_wise_trainer = None
        self.optimizer_state = None
        self.aux_fc_state = None

        os.environ['CUDA_VISIBLE_DEVICES'] = self.settings.gpu

        self.settings.set_save_path()
        write_settings(self.settings)
        self.logger = get_logger(self.settings.save_path, "finetune")
        self.tensorboard_logger = TensorboardLogger(self.settings.save_path)
        self.logger.info("|===>Result will be saved at {}".format(self.settings.save_path))
        self.epoch = 0
        self.test_input = None

        self.prepare()
        self.ma = ModelAnalyse(self.pruned_model,self.logger)

    def write_settings(self):
        """
        Save expriment settings to a file
        """

        with open(os.path.join(self.settings.save_path, "settings.log"), "w") as f:
            for k, v in self.settings.__dict__.items():
                f.write(str(k) + ": " + str(v) + "\n")

    def prepare(self):
        """
        Preparing experiments
        """

        self._set_gpu()
        self._set_dataloader()
        self._set_model()
        self._set_checkpoint()
        self._set_trainier()

    def _set_gpu(self):
        """
        Initialize the seed of random number generator
        """

        # set torch seed
        # init random seed
        torch.manual_seed(self.settings.seed)
        torch.cuda.manual_seed(self.settings.seed)
        torch.cuda.set_device(0)
        cudnn.benchmark = True

    def _set_dataloader(self):
        """
        Create train loader and validation loader for channel pruning
        """

        if 'cifar' in self.settings.dataset:
            self.train_loader, self.val_loader = get_cifar_dataloader(self.settings.dataset,
                                                                      self.settings.batch_size,
                                                                      self.settings.n_threads,
                                                                      self.settings.data_path,
                                                                      self.logger)
        elif self.settings.dataset == 'imagenet':
            self.train_loader, self.val_loader = get_imagenet_dataloader(self.settings.dataset,
                                                                         self.settings.batch_size,
                                                                         self.settings.n_threads,
                                                                         self.settings.data_path,
                                                                         self.logger)

    def _set_model(self):
        """
        Get model
        """

        if self.settings.dataset in ["cifar10", "cifar100"]:
            if self.settings.net_type == "preresnet":
                self.pruned_model = PrunedPreResNet(depth=self.settings.depth,
                                                    pruning_rate=self.settings.pruning_rate,
                                                    num_classes=self.settings.n_classes)
            elif self.settings.net_type == "vgg":
                self.pruned_model = PrunedVGGCIFAR(pruning_rate=self.settings.pruning_rate,
                                                   num_classes=self.settings.n_classes)
            else:
                assert False, "use {} data while network is {}".format(self.settings.dataset, self.settings.net_type)

        elif self.settings.dataset in ["imagenet", "imagenet_mio"]:
            if self.settings.net_type == "resnet":
                self.pruned_model = PrunedResNet(
                    depth=self.settings.depth,
                    pruning_rate=self.settings.pruning_rate,
                    num_classes=self.settings.n_classes)
            else:
                assert False, "use {} data while network is {}".format(self.settings.dataset, self.settings.net_type)

        else:
            assert False, "unsupported data set: {}".format(self.settings.dataset)

    def _set_checkpoint(self):
        """
        Load pre-trained model or resume checkpoint
        """

        assert self.pruned_model is not None, "please create model first"

        self.checkpoint = AuxCheckPoint(self.settings.save_path, self.logger)
        self._load_pretrained()

    def _load_pretrained(self):
        """
        Load pre-trained model
        """

        if self.settings.pretrained is not None:
            check_point_params = torch.load(self.settings.pretrained)
            model_state = check_point_params
            # self.aux_fc_state = check_point_params["aux_fc"]
            self.pruned_model = self.checkpoint.load_state(self.pruned_model, model_state)
            self.logger.info("|===>load restrain file: {}".format(self.settings.pretrained))

    def _set_trainier(self):
        """
        Initialize network-wise trainer
        """

        self.trainer = Trainer(model=self.pruned_model,
                               train_loader=self.train_loader,
                               val_loader=self.val_loader,
                               settings=self.settings,
                               logger=self.logger,
                               tensorboard_logger=self.tensorboard_logger,
                               run_count=self.epoch)

    def evaluation(self):
        """
        Conduct network-wise fine-tuning after channel selection
        """

        self.trainer.val(1)
    def compute_flops(self):
      # s = self.trainer.flops_count(x)
      # return s
      # if test_tensor is None:
      x=torch.randn([128,3,32,32])
      if self.settings.n_gpus == 1:
          x = x.cuda()
      self.pruned_model.eval()
      result = float()
      with torch.no_grad():
        result = self.ma.flops_compute(x)
      return result

In [17]:
path = '/content/DCP/dcp/test/test.hocon'
option = Option(path)
experiment = Experiment(option)

/content/DCP/dcp/test/log_ft_preresnet56_cifar10_bs128_e1_lr0.100_step[160, 240]_evaluation_20200522 / file exist!
Select Action: d (delete) / q (quit):d
|===>Result will be saved at /content/DCP/dcp/test/log_ft_preresnet56_cifar10_bs128_e1_lr0.100_step[160, 240]_evaluation_20200522 /
|===>Result will be saved at /content/DCP/dcp/test/log_ft_preresnet56_cifar10_bs128_e1_lr0.100_step[160, 240]_evaluation_20200522 /
|===>Result will be saved at /content/DCP/dcp/test/log_ft_preresnet56_cifar10_bs128_e1_lr0.100_step[160, 240]_evaluation_20200522 /
|===>Result will be saved at /content/DCP/dcp/test/log_ft_preresnet56_cifar10_bs128_e1_lr0.100_step[160, 240]_evaluation_20200522 /


I0523 08:56:42.236391 140589898045312 <ipython-input-16-b0c383c89702>:47] |===>Result will be saved at /content/DCP/dcp/test/log_ft_preresnet56_cifar10_bs128_e1_lr0.100_step[160, 240]_evaluation_20200522 /


|===>Get datalaoder for cifar10
|===>Get datalaoder for cifar10
|===>Get datalaoder for cifar10
|===>Get datalaoder for cifar10


I0523 08:56:42.253545 140589898045312 dataloader.py:22] |===>Get datalaoder for cifar10


Files already downloaded and verified
|===>load restrain file: /content/resnet_56_pruned0.5.pth
|===>load restrain file: /content/resnet_56_pruned0.5.pth
|===>load restrain file: /content/resnet_56_pruned0.5.pth
|===>load restrain file: /content/resnet_56_pruned0.5.pth


I0523 08:56:43.806663 140589898045312 <ipython-input-16-b0c383c89702>:152] |===>load restrain file: /content/resnet_56_pruned0.5.pth


# 实验测试结果 FLOPS：1.260372e+08

In [19]:
ans = experiment.compute_flops()

|===>FLOPs of layer [0]: 8.683520e+05, 0.006890
|===>FLOPs of layer [0]: 8.683520e+05, 0.006890
|===>FLOPs of layer [0]: 8.683520e+05, 0.006890
|===>FLOPs of layer [0]: 8.683520e+05, 0.006890


I0523 08:56:59.214484 140589898045312 model_analyse.py:196] |===>FLOPs of layer [0]: 8.683520e+05, 0.006890


|===>FLOPs of layer [1]: 2.351104e+06, 0.018654
|===>FLOPs of layer [1]: 2.351104e+06, 0.018654
|===>FLOPs of layer [1]: 2.351104e+06, 0.018654
|===>FLOPs of layer [1]: 2.351104e+06, 0.018654


I0523 08:56:59.223854 140589898045312 model_analyse.py:196] |===>FLOPs of layer [1]: 2.351104e+06, 0.018654


|===>FLOPs of layer [2]: 2.342912e+06, 0.018589
|===>FLOPs of layer [2]: 2.342912e+06, 0.018589
|===>FLOPs of layer [2]: 2.342912e+06, 0.018589
|===>FLOPs of layer [2]: 2.342912e+06, 0.018589


...

I0523 08:56:59.761866 140589898045312 model_analyse.py:196] |===>FLOPs of layer [56]: 2.355200e+06, 0.018687


|===>FLOPs of layer [57]: 1.280000e+03, 0.000010
|===>FLOPs of layer [57]: 1.280000e+03, 0.000010
|===>FLOPs of layer [57]: 1.280000e+03, 0.000010
|===>FLOPs of layer [57]: 1.280000e+03, 0.000010


I0523 08:56:59.768891 140589898045312 model_analyse.py:196] |===>FLOPs of layer [57]: 1.280000e+03, 0.000010


### Total FLOPs: 1.260372e+08
### Total FLOPs: 1.260372e+08
### Total FLOPs: 1.260372e+08
### Total FLOPs: 1.260372e+08


I0523 08:56:59.775609 140589898045312 model_analyse.py:198] ### Total FLOPs: 1.260372e+08
